# Benchmark of EXtra-foam

Author: Jun Zhu, March 03, 2020

In [1]:
from extra_foam.algorithms import mask_image_data, nanmean_image_data, nansum
import random
import numpy as np

In [2]:
import multiprocessing as mp

mp.cpu_count()

72

Assume we want to process a train of detector images.

In [3]:
images = np.random.randn(128, 1200, 1200).astype(np.float32)
images[:, ::2, ::2] = np.nan

### Benchmark nanmean

In [4]:
%timeit np.nanmean(images, axis=0)

1.21 s ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit nanmean_image_data(images)

13.5 ms ± 759 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


A common use case is to apply `nanmean` to a list of selected images.

In [6]:
# select 120 images out of 128 ones
selected = random.sample(range(images.shape[0]), 120)

In [7]:
# numpy is slower than operating on all the images since it copies the data when 'selected' is a list.
%timeit np.nanmean(images[selected], axis=0)

1.43 s ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit nanmean_image_data(images, kept=selected)

12.1 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Benchmark masking

threshold mask

In [9]:
imgs = images.copy()
%timeit imgs[(imgs > 1) | (imgs < -1)] = np.nan

353 ms ± 216 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
imgs = images.copy()
%timeit mask_image_data(imgs, threshold_mask=(-1, 1))

19.6 ms ± 533 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


image mask

In [11]:
image_mask = np.ones((1200, 1200), dtype=bool)
image_mask[::3, ::3] = np.nan

In [12]:
imgs = images.copy()
%timeit imgs[:, image_mask] = np.nan

723 ms ± 424 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
imgs = images.copy()
%timeit mask_image_data(imgs, image_mask=image_mask)

27.5 ms ± 71.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


threshold mask and image mask

In [14]:
imgs = images.copy()
%timeit imgs[(image_mask) | (imgs > 1) | (imgs < -1)] = np.nan

637 ms ± 595 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
imgs = images.copy()
%timeit mask_image_data(imgs, image_mask=image_mask, threshold_mask=(-1, 1))

27.5 ms ± 37.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Benchmark nan-statistics

Non-paralleled funtion can be also much faster than `numpy`.

In [16]:
images = np.random.randn(1200, 1200).astype(np.float32)
images[::2, ::2] = np.nan

In [17]:
%timeit np.nansum(images)

4.64 ms ± 4.84 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit nansum(images)

1.48 ms ± 60.1 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
